In [ ]:
import csv
import os
import random
import blocksci

chain = blocksci.Blockchain("/home/blocksci_data/blocksci.conf")

random.seed(42)

csv_files = os.listdir('csv_data')
for cf in csv_files:
    with open('csv_data/'+cf) as f:
        csv_f = csv.reader(f)
        header = next(csv_f)
        addresses = [row[0] for row in csv_f]
        print(cf, len(addresses))
        if len(addresses) >= 1000:
            addresses = random.sample(addresses, 1000)
            
        print(cf, len(addresses))
        p_tx = n_tx = tx_cnt = 0
        p_file = open('endpoint_addresses/{}_positive.txt'.format(cf[:-4]), "w+")
        n_file = open('endpoint_addresses/{}_negative.txt'.format(cf[:-4]), "w+")
        for addr in addresses:
            address = chain.address_from_string(addr)
            for in_tx in address.input_txes:
                f1 = False
                f2 = False
                for out in in_tx.outs:
                    if str(out.address.first_tx.hash) == str(in_tx.hash):
                        f1 = True
                    else:
                        f2 = True
                    if f1 and f2:
                        break
                if f1:
                    p_file.write('{} {}\n'.format(str(in_tx.hash), addr))
                    p_tx+=1
                if f2:
                    n_file.write('{} {}\n'.format(str(in_tx.hash), addr))
                    n_tx+=1
                tx_cnt+=1
        p_file.close()
        n_file.close()
        print(p_tx, n_tx, tx_cnt)